In [1]:
import pandas as pd

from scripts.data_genertion.consts import *

Added a column of the true length of the sequence according to the string length itself

Removed any rows with NaN inhibition cells

In [8]:
df = pd.read_csv('data_updated_18.5.csv')
df['true_length_of_seq'] = df[SEQUENCE].apply(len)
df = df.dropna(subset=[INHIBITION])

/tmp/ipykernel_6710/2826274691.py:1: DtypeWarning: Columns (20) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data_updated_18.5.csv')


Filling Nan values

In [9]:
df_filled = df.copy()
for col in df.columns:
    if df[col].dtype == 'object':
        df_filled[col] = df[col].fillna("")
    else:
        df_filled[col] = df[col].fillna(0)

This part identify rows that may be the same experiment but with different modifications - in the type, location or pattern. This checks where there are groups of the same 'experiment_keys' that indicate the experiment, and search for more than one unique value at the 'columns_allowed_to_differ' where the modifications or linkage may be different to enable a comparison for only sequences that we can differentiate without considering the modifications. The added column is 'mod_scan', where 1 is a row that has at least one more row with the same experiment values and different modifications or locations of those.

In [11]:
experiment_keys = [SEQUENCE, 'Density(cells/well)', 'Transfection', 'Treatment_Period(hours)', 'ASO_volume(nM)']

columns_allowed_to_differ = [MODIFICATION, 'Location', 'Chemical_Pattern', 'Linkage', 'Linkage_Location', 'Smiles',
                             INHIBITION]

columns_to_compare = [col for col in df_filled.columns if col not in experiment_keys + columns_allowed_to_differ + ['ISIS', 'index']]

result_temp = []
for keys, group in df_filled.groupby(experiment_keys):
    # Check if any of the modification-related columns differ
    if group[[MODIFICATION, 'Location', 'Chemical_Pattern', 'Linkage', 'Linkage_Location', 'Smiles']].nunique().gt(1).any():
        # Check that all other fields (except inhibition + modifications) are the same
        compare = group[columns_to_compare].reset_index(drop=True)
        if compare.nunique().eq(1).all():
            result_temp.append(group)

if result_temp:
    final_df = pd.concat(result_temp)

    # Get the row indices of matched rows
    mod_scan_indices = final_df.index

    # Add new column to original dataframe
    df_filled['mod_scan'] = 0
    df_filled.loc[mod_scan_indices, 'mod_scan'] = 1

    print(df_filled[df_filled['mod_scan'] == 1])  # Optional: preview marked rows
else:
    df_filled['mod_scan'] = 0
    print("No matching ASO experiment groups with only modification-related differences.")


       index    ISIS Target_gene Cell_line  Density(cells/well) Transfection  \
2          2  540806       K-RAS      A431               5000.0  free uptake   
13        13  651966       K-RAS      A431               5000.0  free uptake   
15        15  651987       K-RAS      A431               5000.0  free uptake   
30        30  695867       K-RAS      A431               5000.0  free uptake   
31        31  695912       K-RAS      A431               5000.0  free uptake   
...      ...     ...         ...       ...                  ...          ...   
36867  36867  972044       APOL1      A431              11000.0  free uptake   
36869  36869  972096       APOL1      A431              11000.0  free uptake   
36872  36872  969448       APOL1      A431              11000.0  free uptake   
36873  36873  971969       APOL1      A431              11000.0  free uptake   
36875  36875  972189       APOL1      A431              11000.0  free uptake   

       ASO_volume(nM)  Treatment_Period

In [13]:
df_filled.to_csv('data_asoptimizer_updated.csv', index=False)

Adding a column of correct cell line to fix A431/A-431

In [12]:
df_org = pd.read_csv('data_from_article_fixed.csv')
df_ver2 = pd.read_csv('data_updated_18.5.csv')
df_ver3 = pd.read_csv('data_asoptimizer_updated.csv')

/tmp/ipykernel_1753/2373224261.py:2: DtypeWarning: Columns (20) have mixed types. Specify dtype option on import or set low_memory=False.
  df_ver2 = pd.read_csv('data_updated_18.5.csv')
/tmp/ipykernel_1753/2373224261.py:3: DtypeWarning: Columns (20) have mixed types. Specify dtype option on import or set low_memory=False.
  df_ver3 = pd.read_csv('data_asoptimizer_updated.csv')


In [13]:
df_org['cell_line_uniform'] = df_org['Cell_line']
df_org.loc[(df_org['Cell_line'] == 'A431') | (df_org['Cell_line'] == 'A-431'), 'cell_line_uniform'] = 'A431'

In [15]:
df_ver2['cell_line_uniform'] = df_ver2['Cell_line']
df_ver2.loc[(df_ver2['Cell_line'] == 'A431') | (df_ver2['Cell_line'] == 'A-431'), 'cell_line_uniform'] = 'A431'

In [16]:
df_ver3['cell_line_uniform'] = df_ver3['Cell_line']
df_ver3.loc[(df_ver3['Cell_line'] == 'A431') | (df_ver3['Cell_line'] == 'A-431'), 'cell_line_uniform'] = 'A431'

In [ ]:
df_org.to_csv('data_from_article_fixed.csv', index=False)
df_ver2.to_csv('data_updated_18.5.csv', index=False)
df_ver3.to_csv('data_asoptimizer_updated.csv', index=False)

**Analyzing invalid rows and single experiment rows**

In [2]:
df = pd.read_csv('data_asoptimizer_updated.csv')

/tmp/ipykernel_7384/671419099.py:1: DtypeWarning: Columns (20) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data_asoptimizer_updated.csv')


In [3]:
df_copy = df.copy()
df_copy['cell_line_check'] = 0
df_copy.loc[(df_copy['Cell line organism'] == 'unknown'), 'cell_line_check'] = 1

In [6]:
volumes = df_copy['ASO_volume(nM)'].unique()
volumes.sort()

In [7]:
df_copy['aso_volume_high_check'] = 0
df_copy.loc[(df_copy['ASO_volume(nM)']  >= 10000.0), 'aso_volume_high_check'] = 1

In [8]:
df_copy[(df_copy['aso_volume_high_check'] == 1) | (df_copy['cell_line_check'] == 1)]

,index,ISIS,Target_gene,Cell_line,Density(cells/well),Transfection,ASO_volume(nM),Treatment_Period(hours),Primer_probe_set,Sequence,...,Canonical Gene Name,Cell line organism,Transcript,Location_in_sequence,Location_div_by_length,true_length_of_seq,mod_scan,cell_line_uniform,cell_line_check,aso_volume_high_check
2576,2576,709897,SNCA,Human IPS,35000.0,free uptake,20000.0,168,RTS2621,TTGAAATTTTCCAGCTA,...,SNCA,human,NaN,47414.0,0.343652,17,0,Human IPS,0,1
2577,2577,740425,SNCA,Human IPS,35000.0,free uptake,20000.0,168,RTS2621,ATGAATACATCCATGGC,...,SNCA,human,NaN,2645.0,0.019171,17,0,Human IPS,0,1
2578,2578,741082,SNCA,Human IPS,35000.0,free uptake,20000.0,168,RTS2621,TTTTTGTTAATAGTTCT,...,SNCA,human,NaN,27393.0,0.198542,17,0,Human IPS,0,1
2579,2579,788815,SNCA,Human IPS,35000.0,free uptake,20000.0,168,RTS2621,TTCCTTTACACCACACT,...,SNCA,human,NaN,2621.0,0.018997,17,1,Human IPS,0,1
2580,2580,788816,SNCA,Human IPS,35000.0,free uptake,20000.0,168,RTS2621,ATTCCTTTACACCACAC,...,SNCA,human,NaN,2622.0,0.019004,17,1,Human IPS,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32620,34745,495819,DGAT2,HepG2,20000.0,Lipofectin reagent,10000.0,16,RTS2988_MGB,TTTCCATTTTTTACCCAACC,...,DGAT2,human,NaN,29440.0,0.700552,20,0,HepG2,0,1
32621,34746,495825,DGAT2,HepG2,20000.0,Lipofectin reagent,10000.0,16,RTS2988_MGB,CATACTAGACTATGCAGCCT,...,DGAT2,human,NaN,32046.0,0.762564,20,0,HepG2,0,1
32622,34747,495832,DGAT2,HepG2,20000.0,Lipofectin reagent,10000.0,16,RTS2988_MGB,GTTGCCATCATACTAGACTA,...,DGAT2,human,NaN,32054.0,0.762755,20,0,HepG2,0,1
32623,34748,495809,DGAT2,HepG2,20000.0,Lipofectin reagent,10000.0,16,RTS2988_MGB,CCCAACCTTTAAGGCCTATC,...,DGAT2,human,NaN,29427.0,0.700243,20,0,HepG2,0,1


In [9]:
df_copy[df_copy['Density(cells/well)'] == 0]
df_copy['density_zero_check'] = 0
df_copy.loc[(df_copy['Density(cells/well)']  == 0), 'density_zero_check'] = 1

In [10]:
unique_cell_line = df_copy['Cell_line'].unique()
cell_line_counts = df_copy['Cell_line'].value_counts().reindex(unique_cell_line, fill_value=0)

In [11]:
df_copy['invalidating'] = 0
df_copy.loc[(df_copy['Cell_line'] == 'LLC-MK2 monkey'), 'invalidating'] = 1

In [12]:
unique_seq = df_copy['Sequence'].unique()
seq_counts = df_copy['Sequence'].value_counts().reindex(unique_seq, fill_value=0)

In [13]:
seq_counts[seq_counts == 1]

Sequence
GCTAAAACAAATGCTA        1
TATAATGGTGAATATC        1
GGTGAATATCTTCAAA        1
CACTTGTACTAGTATG        1
GCATTGCTAGTTCAAA        1
                       ..
GCTATTCATTTTTCTGAGCC    1
AGAGGCCCTGGACACTGGCC    1
TCAGCCTAGAGGCCCTGGAC    1
CCAACAGTGGTGATGGGCTT    1
GCTTACCAACAGTGGTGATG    1
Name: count, Length: 11502, dtype: int64

In [14]:
from collections import defaultdict

def is_shift_similar(seq1, seq2, min_overlap=10):
    len1, len2 = len(seq1), len(seq2)
    min_len = min(len1, len2)

    for shift in range(-min_len + min_overlap, min_len - min_overlap + 1):
        if shift < 0:
            overlap1 = seq1[-shift:]
            overlap2 = seq2[:len(seq1)+shift]
        else:
            overlap1 = seq1[:len(seq2)-shift]
            overlap2 = seq2[shift:]

        if len(overlap1) >= min_overlap and overlap1 == overlap2:
            return True
    return False

def build_similarity_graph(sequences, min_overlap=10):
    graph = defaultdict(list)
    for i in range(len(sequences)):
        for j in range(i + 1, len(sequences)):
            if is_shift_similar(sequences[i], sequences[j], min_overlap=min_overlap):
                graph[i].append(j)
                graph[j].append(i)
    return graph

def find_connected_components(graph, num_nodes):
    visited = set()
    components = []

    def dfs(node, group):
        visited.add(node)
        group.append(node)
        for neighbor in graph[node]:
            if neighbor not in visited:
                dfs(neighbor, group)

    for node in range(num_nodes):
        if node not in visited:
            group = []
            dfs(node, group)
            components.append(group)

    return components

In [15]:
sequences = unique_seq
graph = build_similarity_graph(sequences, min_overlap=10)
groups = find_connected_components(graph, len(sequences))

single_group = []

for group in groups:
    print("Similar group:")
    for idx in group:
        print(f"  {sequences[idx]}")
        if len(group) == 1:
            single_group.append(sequences[idx])

Similar group:
  GCTAAAACAAATGCTA
  ACAAATGCTAATAATT
  TGCTAATAATTTAGTG
  CAAATGCTAATAATTT
  GCTAATAATTTAGTGT
  ATGCTAATAATTTAGT
  AATGCTAATAATTTAG
  AAATGCTAATAATTTA
  AACAAATGCTAATAAT
  AAACAAATGCTAATAA
  AAAACAAATGCTAATA
  TAAAACAAATGCTAAT
  CTAAAACAAATGCTAA
Similar group:
  TATAATGGTGAATATC
  GGTGAATATCTTCAAA
  TGGTGAATATCTTCAA
  AATGGTGAATATCTTC
  GTGAATATCTTCAAAT
  ATGGTGAATATCTTCA
  TAATGGTGAATATCTT
  ATAATGGTGAATATCT
Similar group:
  GCATGAAGATTTCTGG
Similar group:
  CACTTGTACTAGTATG
  GTACTAGTATGCCTTA
  TTGTACTAGTATGCCT
  TACTAGTATGCCTTAA
  TGTACTAGTATGCCTT
  CTTGTACTAGTATGCC
  ACTTGTACTAGTATGC
Similar group:
  GCATTGCTAGTTCAAA
  AGGCATTGCTAGTTCA
  CATTGCTAGTTCAAAA
  GGCATTGCTAGTTCAA
  GTTACCGCATTGCTAG
  CGCATTGCTAGATTCT
  ACCGCATTGCTAGATT
  GAGTTACCGCATTGCT
  CCTGAGTTACCGCATT
  GGCCTGAGTTACCGCA
  GAAGGCCTGAGTTACC
  CTGAGTTACCGCATTG
  TTACCGCATTGCTAGA
  CATTGCTAGATTCTCC
  TAGATTCTCCCTGCAC
  GCTAGATTCTCCCTGC
  TTGCTAGATTCTCCCT
  CCGCATTGCTAGATTC
  TGCTAGATTCTCCCTG
  CTAGATTCTCC

In [16]:
single_group

['GCATGAAGATTTCTGG',
 'AGTCTATTTCAGGCGG',
 'AGCTTGATCGAAGAGT',
 'GGCTACTACGCCGTCA',
 'GGTAGTTCCAACGATGT',
 'TTTTCAGCACCCATGGG',
 'GTGACTTTTAGAAATGA',
 'CTCATGAATACATATAA',
 'TTCTATGGTAACCATCC',
 'TAGTGTAAGATGACACA',
 'ACTGTTCAATAACAAAT',
 'TCCTCTATTTCTTAATT',
 'TAAATTCATGGTCACAA',
 'AAAATTACCGTCAGATA',
 'AGGCTTATATGACTTAA',
 'ACCGTGGAGTCATATGA',
 'ACACATTAGATTGTTCT',
 'GAAACATGTTTGCATCT',
 'ACCAAGAGAGCGGGCAG',
 'AAAGAATGCCACTAGGC',
 'TACAGGTGCAGTTATAT',
 'GACATCTCTAACATAAA',
 'GATGGATATTGACTCCT',
 'TATATGCATTTTTCAGG',
 'AGACACTCTTACTTGAG',
 'TGAAGGACAACTGTGTA',
 'GACATCTGAAGTGTTCA',
 'ATAACCACCACTGAATT',
 'CCATGCTACATTGCTCA',
 'GTCCCCAATCCCCACCC',
 'TCCTTGAAAACTTCCAT',
 'GAGGTCAAATTTTCCAG',
 'GAGTGACAGTGGTGGGC',
 'TGAGTTAAACAAAGTTA',
 'AAGGTGACTCTGGTAGT',
 'CATATATTTGGCAACAT',
 'CCATCAAGTTTATTTTC',
 'ACTTTCTACTAGTGACT',
 'ATATCACATTACTCATG',
 'GTAAAAAAGGAAGTTTC',
 'CCATTTCTCTCTAGTGT',
 'TCCTGAAATATACTGTT',
 'GTCTAGTTCTGTCCTCT',
 'CACATAAATCCTTAAAT',
 'TTCACTGCTCAAAAATT',
 'GCTTCCTGAAAA

In [23]:
# Count how many times each sequence appears in the full df_copy
sequence_counts = df_copy['Sequence'].value_counts()

# Filter sequences in single_group that appear only once
truly_unique_sequences = [seq for seq in single_group if sequence_counts.get(seq, 0) == 1]

# Create the final filtered DataFrame
df_final_single_seq = df_copy[df_copy['Sequence'].isin(truly_unique_sequences)]

In [24]:
df_final_single_seq

,index,ISIS,Target_gene,Cell_line,Density(cells/well),Transfection,ASO_volume(nM),Treatment_Period(hours),Primer_probe_set,Sequence,...,Transcript,Location_in_sequence,Location_div_by_length,true_length_of_seq,mod_scan,cell_line_uniform,cell_line_check,aso_volume_high_check,density_zero_check,invalidating
679,679,651837,K-RAS,A431,5000.0,free uptake,5000.0,72,RTS3496_MGB,AGTCTATTTCAGGCGG,...,NaN,1295.0,0.028343,16,0,A431,0,0,0,0
680,680,696241,K-RAS,A431,5000.0,free uptake,5000.0,72,RTS3496_MGB,AGCTTGATCGAAGAGT,...,NaN,43931.0,0.961480,16,0,A431,0,0,0,0
980,980,709734,SNCA,SH-SY5Y,20000.0,electroporation,7000.0,24,RTS2621,GGTAGTTCCAACGATGT,...,NaN,113014.0,0.819114,17,0,SH-SY5Y,0,0,0,0
982,982,709746,SNCA,SH-SY5Y,20000.0,electroporation,7000.0,24,RTS2621,TTTTCAGCACCCATGGG,...,NaN,113166.0,0.820216,17,0,SH-SY5Y,0,0,0,0
983,983,709752,SNCA,SH-SY5Y,20000.0,electroporation,7000.0,24,RTS2621,GTGACTTTTAGAAATGA,...,NaN,113262.0,0.820912,17,0,SH-SY5Y,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30515,32640,413447,DGAT2,HepG2,10000.0,Lipofectin reagent,120.0,24,RTS2367,TGTCCTTATCAAAATCCCTC,...,NaN,35167.0,0.836831,20,0,HepG2,0,0,0,0
30519,32644,413451,DGAT2,HepG2,10000.0,Lipofectin reagent,120.0,24,RTS2367,GAGGCTCAGGCCTTGACAAC,...,NaN,37056.0,0.881782,20,0,HepG2,0,0,0,0
30520,32645,413452,DGAT2,HepG2,10000.0,Lipofectin reagent,120.0,24,RTS2367,TGTTATCCGAGTTGAATTCT,...,NaN,37203.0,0.885280,20,0,HepG2,0,0,0,0
30521,32646,413453,DGAT2,HepG2,10000.0,Lipofectin reagent,120.0,24,RTS2367,GTTTTGGGAACTCATGCATT,...,NaN,37417.0,0.890372,20,0,HepG2,0,0,0,0


In [48]:
columns_to_check = ['Canonical Gene Name', 'cell_line_uniform', 'Density(cells/well)', 'Transfection', 'ASO_volume(nM)', 'Treatment_Period(hours)', 'Primer_probe_set']
df_unique_combos = df_final_single_seq[df_final_single_seq.duplicated(subset=columns_to_check, keep=False) == False]

In [36]:
df_unique_combos['index']
df_copy.loc[(df_copy['index'] == 28908), 'invalidating'] = 1

In [41]:
columns_to_check.append('Modification')
columns_to_check.append('Linkage')

In [49]:
rare_value_mask = pd.Series(False, index=df_copy.index)

# Loop through each column
for col in columns_to_check:
    # Count how many times each value appears
    value_counts = df_copy[col].value_counts()

    # Get the values that occur less than 5 times
    rare_values = value_counts[value_counts < 5].index

    # Update the mask: True if this row has a rare value in this column
    rare_value_mask |= df_copy[col].isin(rare_values)

# Now get all rows that have a rare value in any of the columns
df_with_rare_values = df_copy[rare_value_mask]

In [50]:
df_with_rare_values

,index,ISIS,Target_gene,Cell_line,Density(cells/well),Transfection,ASO_volume(nM),Treatment_Period(hours),Primer_probe_set,Sequence,...,Transcript,Location_in_sequence,Location_div_by_length,true_length_of_seq,mod_scan,cell_line_uniform,cell_line_check,aso_volume_high_check,density_zero_check,invalidating
897,897,549148,negative_control,A431,5000.0,free uptake,37.0,72,RTS3496_MGB,GGCTACTACGCCGTCA,...,NaN,0.0,0.000000,16,0,A431,0,0,0,0
906,906,549148,negative_control,A431,5000.0,free uptake,37.0,72,RTS3496_MGB,GGCTACTACGCCGTCA,...,NaN,0.0,0.000000,16,0,A431,0,0,0,0
2610,2610,788833,SNCA,LLC-MK2 monkey,20000.0,electroporation,4000.0,24,RTS2621,AATTCCTTTACACCACA,...,NaN,2623.0,0.019011,17,0,LLC-MK2 monkey,0,0,0,1
2611,2611,788833,SNCA,LLC-MK2 monkey,20000.0,electroporation,20000.0,24,RTS2621,AATTCCTTTACACCACA,...,NaN,2623.0,0.019011,17,0,LLC-MK2 monkey,0,1,0,1


In [54]:
df_copy.loc[(df_copy['Target_gene'] == 'negative_control'), 'invalidating'] = 1

In [62]:
df_copy.to_csv('data_with_flags.csv', index=False)

**Checking after analyzing**

In [2]:
df_check = pd.read_csv('data_with_flags.csv')

/tmp/ipykernel_4942/83387656.py:1: DtypeWarning: Columns (20) have mixed types. Specify dtype option on import or set low_memory=False.
  df_check = pd.read_csv('data_with_flags.csv')


In [3]:
df_check[df_check['aso_volume_high_check'] == 1]

,index,ISIS,Target_gene,Cell_line,Density(cells/well),Transfection,ASO_volume(nM),Treatment_Period(hours),Primer_probe_set,Sequence,...,Transcript,Location_in_sequence,Location_div_by_length,true_length_of_seq,mod_scan,cell_line_uniform,cell_line_check,aso_volume_high_check,density_zero_check,invalidating
2576,2576,709897,SNCA,Human IPS,35000.0,free uptake,20000.0,168,RTS2621,TTGAAATTTTCCAGCTA,...,NaN,47414.0,0.343652,17,0,Human IPS,0,1,0,0
2577,2577,740425,SNCA,Human IPS,35000.0,free uptake,20000.0,168,RTS2621,ATGAATACATCCATGGC,...,NaN,2645.0,0.019171,17,0,Human IPS,0,1,0,0
2578,2578,741082,SNCA,Human IPS,35000.0,free uptake,20000.0,168,RTS2621,TTTTTGTTAATAGTTCT,...,NaN,27393.0,0.198542,17,0,Human IPS,0,1,0,0
2579,2579,788815,SNCA,Human IPS,35000.0,free uptake,20000.0,168,RTS2621,TTCCTTTACACCACACT,...,NaN,2621.0,0.018997,17,1,Human IPS,0,1,0,0
2580,2580,788816,SNCA,Human IPS,35000.0,free uptake,20000.0,168,RTS2621,ATTCCTTTACACCACAC,...,NaN,2622.0,0.019004,17,1,Human IPS,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32620,34745,495819,DGAT2,HepG2,20000.0,Lipofectin reagent,10000.0,16,RTS2988_MGB,TTTCCATTTTTTACCCAACC,...,NaN,29440.0,0.700552,20,0,HepG2,0,1,0,0
32621,34746,495825,DGAT2,HepG2,20000.0,Lipofectin reagent,10000.0,16,RTS2988_MGB,CATACTAGACTATGCAGCCT,...,NaN,32046.0,0.762564,20,0,HepG2,0,1,0,0
32622,34747,495832,DGAT2,HepG2,20000.0,Lipofectin reagent,10000.0,16,RTS2988_MGB,GTTGCCATCATACTAGACTA,...,NaN,32054.0,0.762755,20,0,HepG2,0,1,0,0
32623,34748,495809,DGAT2,HepG2,20000.0,Lipofectin reagent,10000.0,16,RTS2988_MGB,CCCAACCTTTAAGGCCTATC,...,NaN,29427.0,0.700243,20,0,HepG2,0,1,0,0


In [4]:
seq = 'ACtTtatttccaaattcactTtTAC'
seq = seq.upper()
df_check[df_check['Sequence'] == seq]

,index,ISIS,Target_gene,Cell_line,Density(cells/well),Transfection,ASO_volume(nM),Treatment_Period(hours),Primer_probe_set,Sequence,...,Transcript,Location_in_sequence,Location_div_by_length,true_length_of_seq,mod_scan,cell_line_uniform,cell_line_check,aso_volume_high_check,density_zero_check,invalidating
24492,24502,50002020,Tau,Tubulin,0.0,NaN,5000.0,24,NaN,ACTTTATTTCCAAATTCACTTTTAC,...,NaN,133912.0,0.999694,25,1,Tubulin,1,0,1,0
24497,24507,50002086,Tau,Tubulin,0.0,NaN,5000.0,24,NaN,ACTTTATTTCCAAATTCACTTTTAC,...,NaN,133912.0,0.999694,25,1,Tubulin,1,0,1,0
24498,24508,50002029,Tau,Tubulin,0.0,NaN,5000.0,24,NaN,ACTTTATTTCCAAATTCACTTTTAC,...,NaN,133912.0,0.999694,25,1,Tubulin,1,0,1,0
24500,24510,50002095,Tau,Tubulin,0.0,NaN,5000.0,24,NaN,ACTTTATTTCCAAATTCACTTTTAC,...,NaN,133912.0,0.999694,25,1,Tubulin,1,0,1,0
24501,24511,50002059,Tau,Tubulin,0.0,NaN,5000.0,24,NaN,ACTTTATTTCCAAATTCACTTTTAC,...,NaN,133912.0,0.999694,25,1,Tubulin,1,0,1,0
24502,24512,50002077,Tau,Tubulin,0.0,NaN,5000.0,24,NaN,ACTTTATTTCCAAATTCACTTTTAC,...,NaN,133912.0,0.999694,25,1,Tubulin,1,0,1,0
24503,24513,50002021,Tau,Tubulin,0.0,NaN,5000.0,24,NaN,ACTTTATTTCCAAATTCACTTTTAC,...,NaN,133912.0,0.999694,25,1,Tubulin,1,0,1,0
24505,24515,50002068,Tau,Tubulin,0.0,NaN,5000.0,24,NaN,ACTTTATTTCCAAATTCACTTTTAC,...,NaN,133912.0,0.999694,25,1,Tubulin,1,0,1,0
24646,24656,50002049,Tau,Tubulin,0.0,NaN,5000.0,24,NaN,ACTTTATTTCCAAATTCACTTTTAC,...,NaN,133912.0,0.999694,25,1,Tubulin,1,0,1,0
24647,24657,50002019,Tau,Tubulin,0.0,NaN,5000.0,24,NaN,ACTTTATTTCCAAATTCACTTTTAC,...,NaN,133912.0,0.999694,25,1,Tubulin,1,0,1,0


In [ ]:
df_check[df_check['cell_line_check'] == 1]

Adding correcting inhibition column and mutual index column for identical rows

In [2]:
df_org = pd.read_csv('data_asoptimizer_updated.csv')

/tmp/ipykernel_6357/2090144437.py:1: DtypeWarning: Columns (20) have mixed types. Specify dtype option on import or set low_memory=False.
  df_org = pd.read_csv('data_asoptimizer_updated.csv')


In [3]:
df_org['Transcript'] = df_org['Transcript'].fillna(0)
df_org['Transcript'] = df_org['Transcript'].astype(str)
df_org['Transfection'] = df_org['Transfection'].fillna(0)
df_org['Transfection'] = df_org['Transfection'].astype(str)
df_org['Primer_probe_set'] = df_org['Primer_probe_set'].fillna(0)
df_org['Primer_probe_set'] = df_org['Primer_probe_set'].astype(str)

In [5]:
def process_duplicate_inhibition(df):
    """
    Process dataframe to handle duplicate rows (identical except for inhibition and index).

    Parameters:
    df (pd.DataFrame): Input dataframe with columns including 'inhibition' and 'index'

    Returns:
    pd.DataFrame: Processed dataframe with 'corrected_inhibition' and 'mutual_index' columns
    """

    # Create a copy to avoid modifying the original dataframe
    result_df = df.copy()

    # Get all columns except 'inhibition' and 'index' for grouping
    grouping_cols = [col for col in df.columns if col not in [INHIBITION, 'index']]

    # Group by all columns except inhibition and index
    grouped = df.groupby(grouping_cols)

    # Initialize the new columns
    result_df['corrected_inhibition'] = df[INHIBITION]
    result_df['mutual_index'] = df['index']

    # Process each group
    for name, group in grouped:
        if len(group) > 1:  # This group has duplicates
            # Calculate average inhibition for duplicates
            avg_inhibition = group[INHIBITION].mean()

            # Get the first index as the mutual index
            mutual_idx = group['index'].iloc[0]

            # Update all rows in this group
            mask = df.index.isin(group.index)
            result_df.loc[mask, 'corrected_inhibition'] = avg_inhibition
            result_df.loc[mask, 'mutual_index'] = mutual_idx

    return result_df

In [6]:
df_copy = process_duplicate_inhibition(df_org)

In [7]:
df_copy

,index,ISIS,Target_gene,Cell_line,Density(cells/well),Transfection,ASO_volume(nM),Treatment_Period(hours),Primer_probe_set,Sequence,...,Canonical Gene Name,Cell line organism,Transcript,Location_in_sequence,Location_div_by_length,true_length_of_seq,mod_scan,cell_line_uniform,corrected_inhibition,mutual_index
0,0,540733,K-RAS,A431,5000.0,free uptake,2000.0,24,RTS3496_MGB,GCTAAAACAAATGCTA,...,KRAS,human,0,41212.0,0.901972,16,0,A431,33.0,0
1,1,540747,K-RAS,A431,5000.0,free uptake,2000.0,24,RTS3496_MGB,TATAATGGTGAATATC,...,KRAS,human,0,23686.0,0.518395,16,0,A431,7.0,1
2,2,540806,K-RAS,A431,5000.0,free uptake,2000.0,24,RTS3496_MGB,GCATGAAGATTTCTGG,...,KRAS,human,0,43363.0,0.949049,16,1,A431,61.0,2
3,3,651479,K-RAS,A431,5000.0,free uptake,2000.0,24,RTS3496_MGB,GGTGAATATCTTCAAA,...,KRAS,human,0,23680.0,0.518264,16,0,A431,28.0,3
4,4,651490,K-RAS,A431,5000.0,free uptake,2000.0,24,RTS3496_MGB,CACTTGTACTAGTATG,...,KRAS,human,0,41168.0,0.901009,16,0,A431,36.0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34760,36885,905505,APOL1,A431,10000.0,free uptake,40.0,16,RTS35962,GTTATGAAATTATTGG,...,APOL1,human,0,7827.0,0.541286,16,0,A431,52.0,36876
34761,36886,905634,APOL1,A431,10000.0,free uptake,40.0,16,RTS35962,ATTCTATTAGAGGGCT,...,APOL1,human,0,8250.0,0.570539,16,0,A431,29.5,36878
34762,36887,905665,APOL1,A431,10000.0,free uptake,40.0,16,RTS35962,GCTTTAAACTCAGGTG,...,APOL1,human,0,8335.0,0.576418,16,0,A431,11.0,36887
34763,36888,972163,APOL1,A431,10000.0,free uptake,40.0,16,RTS35962,CGTCAATATATTCTTT,...,APOL1,human,0,13848.0,0.957676,16,0,A431,36.0,36888


In [8]:
cols_to_check = [col for col in df_copy.columns if col not in [INHIBITION, 'index', 'mutual_index', 'corrected_inhibition']]

# Find rows that have duplicates ignoring the 'index' column
dups = df_copy.duplicated(subset=cols_to_check, keep=False)

# Filter those rows, keeping the 'index' column
df_duplicates = df_copy[dups]

In [9]:
df_duplicates_sorted = df_duplicates.sort_values(by=cols_to_check).reset_index(drop=True)

In [10]:
df_duplicates_sorted

,index,ISIS,Target_gene,Cell_line,Density(cells/well),Transfection,ASO_volume(nM),Treatment_Period(hours),Primer_probe_set,Sequence,...,Canonical Gene Name,Cell line organism,Transcript,Location_in_sequence,Location_div_by_length,true_length_of_seq,mod_scan,cell_line_uniform,corrected_inhibition,mutual_index
0,33531,217317,DGAT2,HepG2,20000.0,Lipofectin reagent,2500.0,16,RTS2988_MGB,CAGCAGGTTGTGTGTCTTCA,...,DGAT2,human,0,36820.0,0.876166,20,0,HepG2,27.5,33531
1,33579,217317,DGAT2,HepG2,20000.0,Lipofectin reagent,2500.0,16,RTS2988_MGB,CAGCAGGTTGTGTGTCTTCA,...,DGAT2,human,0,36820.0,0.876166,20,0,HepG2,27.5,33531
2,34450,217317,DGAT2,HepG2,20000.0,Lipofectin reagent,10000.0,16,RTS2988_MGB,CAGCAGGTTGTGTGTCTTCA,...,DGAT2,human,0,36820.0,0.876166,20,0,HepG2,71.0,34450
3,34470,217317,DGAT2,HepG2,20000.0,Lipofectin reagent,10000.0,16,RTS2988_MGB,CAGCAGGTTGTGTGTCTTCA,...,DGAT2,human,0,36820.0,0.876166,20,0,HepG2,71.0,34450
4,32906,217317,DGAT2,HepG2,20000.0,Lipofectin reagent,20000.0,16,RTS2988_MGB,CAGCAGGTTGTGTGTCTTCA,...,DGAT2,human,0,36820.0,0.876166,20,0,HepG2,79.5,32906
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4924,29937,80280157,MYH7,CC-2580,0.0,0,5000.0,24,ddPCR,CTTGGCAATGATCTCATCC,...,MYH7,human,0,11969.0,0.522185,19,1,CC-2580,63.5,29776
4925,29777,80280158,MYH7,CC-2580,0.0,0,5000.0,24,ddPCR,CTTGGCAATGATCTCATCC,...,MYH7,human,0,11969.0,0.522185,19,1,CC-2580,43.0,29777
4926,29938,80280158,MYH7,CC-2580,0.0,0,5000.0,24,ddPCR,CTTGGCAATGATCTCATCC,...,MYH7,human,0,11969.0,0.522185,19,1,CC-2580,43.0,29777
4927,29778,80280159,MYH7,CC-2580,0.0,0,5000.0,24,ddPCR,CTTGGCAATGATCTCATCC,...,MYH7,human,0,11969.0,0.522185,19,1,CC-2580,28.5,29778


In [11]:
df_copy.to_csv('data_updated_inhibition.csv', index=False)

In [3]:
df_temp = pd.read_csv('data_asoptimizer_05_08.csv')
df_temp['cell_line_after_check'] = df_temp['cell_line_uniform']

/tmp/ipykernel_5638/1805412292.py:1: DtypeWarning: Columns (29) have mixed types. Specify dtype option on import or set low_memory=False.
  df_temp = pd.read_csv('data_asoptimizer_05_08.csv')


In [4]:
df_temp.loc[df_temp['cell_line_uniform'] == 'Tubulin', 'cell_line_after_check'] = 'mouse primary cells'
df_temp.loc[df_temp['cell_line_uniform'] == 'Tubulin', 'Cell line organism'] = 'mouse'
df_temp.loc[df_temp['cell_line_uniform'] == 'Oscillation', 'cell_line_after_check'] = 'rats primary cells'
df_temp.loc[df_temp['cell_line_uniform'] == 'Oscillation', 'Cell line organism'] = 'rats'
df_temp.loc[df_temp['cell_line_uniform'] == 'SK cells asyn', 'cell_line_after_check'] = 'SK-N-BE(2)'
df_temp.loc[df_temp['cell_line_uniform'] == 'SK cells asyn', 'Cell line organism'] = 'human'
df_temp.loc[df_temp['cell_line_uniform'] == 'PAC neurons asyn', 'cell_line_after_check'] = 'A53T-PAC cells'
df_temp.loc[df_temp['cell_line_uniform'] == 'PAC neurons asyn', 'Cell line organism'] = 'mouse'
df_temp.loc[df_temp['cell_line_uniform'] == 'Angptl2/Actin', 'cell_line_after_check'] = 'SK-N-AS'
df_temp.loc[df_temp['cell_line_uniform'] == 'Angptl2/Actin', 'Cell line organism'] = 'human'

In [6]:
df_temp.to_csv('data_asoptimizer_05_08.csv')